In [1]:
import numpy as np
import os
import shutil
import csv
import pandas as pd
import gzip

In [2]:
dataPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/gdc_download_HTSeq' #zipped file path
dataNewFolder = '/Users/ursulawidocki/Desktop/BreCaNet/Data/processedTCGA_HTSeq'

In [3]:
# Unzips the expression data dn creates a folder (processedTCGA_832) of the unzipped files 
# Gets the codes of the patient folders and expression files

folderIDs = list() # codes of the patient IDs on the folder
indivList = list() # codes of the patients' expression file

for fileName in os.listdir(dataPath):
    if fileName == '.DS_Store' or fileName == 'MANIFEST.txt':
        continue
        
    else:
        folderIDs.append(fileName)
        
    newPath = dataPath + '/' + fileName
    
    # goes into the individual folder
    for indivName in os.listdir(newPath):
        if (indivName == '.DS_Store') or (indivName == 'MANIFEST.txt') or (indivName == 'annotations.txt'):
            continue
        filePath = newPath + '/' + indivName
        
        fileEnding = indivName[-3] + indivName[-2] + indivName[-1]
        if(fileEnding == '.gz'):
            with gzip.open(filePath, 'rb') as f_in:
                newFilePath = dataNewFolder + '/' + indivName[0:-3]
                indivList.append(indivName[0:-3])
                
                with open(newFilePath, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

In [11]:
## Makes file of folder names

folderIDPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/foldIDList_HTSeq.txt'
folderFile = open(folderIDPath, "w+")
for i in range(0, 832):
    newLineFolder = folderIDs[i] + "\n"
    
    folderFile.write(newLineFolder)
    
folderFile.close()

832


In [5]:
## Gets folder codes from expression data
folderIDPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/foldIDList_HTSeq.txt'
folders = pd.read_csv(folderIDPath, delimiter = '\t', header = None)
folders_list = list(folders[0])

In [7]:
## Reads in metadata
# gets file_id and case_id

metaPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/metadata.cart.HTSeq.json' # the same

file_id = list()
f = 1
case_id = list()
temp_case_len = 0
old_case_len = 0
with open(metaPath, "r") as file:
    for line in file.readlines():
            
        
        if("case_id" in line) and (line.strip().split("\"")[3] not in case_id):
            old_case_len = len(case_id)
            case = line.strip().split("\"")[3]
            case_id.append(case)
            temp_case_len = len(case_id)
            
        if ("file_id" in line) and (old_case_len < temp_case_len):
            item = line.strip().split("\"")[3]
            file_id.append(item)
            old_case_len = len(case_id)

print(len(file_id))
print(len(set(case_id)))

714
714


In [8]:
## Gets uuid and barcodes of the patients from clinical data

# reads in clinical information to link patients with their barcode
clinicalPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/gdc_download_clinical_832/8162d394-8b64-4da2-9f5b-d164c54b9608/nationwidechildrens.org_clinical_patient_brca.txt'
clinical_df = pd.read_csv(clinicalPath, delimiter = '\t')

# removes rows that are not necessary to match data
clinical_df = clinical_df.drop(clinical_df.index[0])
clinical_df = clinical_df.drop(clinical_df.index[0])
clinical_df.head()
codes_df = clinical_df[['bcr_patient_uuid', 'bcr_patient_barcode']]
#codes_df.head()

uuid_list = list(codes_df['bcr_patient_uuid'])
barcode_list = list(codes_df['bcr_patient_barcode'])

print(len(uuid_list))
print(len(barcode_list))

1097
1097


In [9]:
## Reads in PAM50 labels

subtype_path = "/Users/ursulawidocki/Desktop/BreCaNet/Code/TCGA_subtypes.txt"
subtypes = pd.read_csv(subtype_path, delimiter = " ")
subtypes.head()

pt_barcodes_list = list(subtypes['brca_subtypes.patient'])
subtype_list = list(subtypes['brca_subtypes.BRCA_Subtype_PAM50'])

print(len(pt_barcodes_list))
print(len(subtype_list))

1087
1087


## Matches

In [10]:
## First, match the folder with the file_ids to get the corresponding cases from the metadata

match_cases = list()

for folder in folders_list:
    for file in file_id:
        if folder == file:
            ind = file_id.index(folder)
            match_cases.append(case_id[ind])

print(len(match_cases)) # should be equivalent to number of samples

714


In [112]:
## Matches case_ids from metadata with uuid and patient barcodes from clinical data

match_barcodes = list()

for case in match_cases:
    for ptID in uuid_list:
        if case == ptID.lower():
            ind = uuid_list.index(ptID)
            match_barcodes.append(barcode_list[ind])
            break
            
print(len(match_barcodes))

714


In [113]:
## Matches subtypes to their barcodes from the TCGA data

match_subtypes = list()

for barcode in match_barcodes:
    ind = pt_barcodes_list.index(barcode)
    match_subtypes.append(subtype_list[ind])
    
print(len(match_subtypes))

714


In [114]:
print(len(set(folders_list)))
print(len(set(file_id)))
print(len(set(case_id)))



print(len(set(match_subtypes))) # there is only this many subtypes among my samples
print((set(match_subtypes)))

832
714
714
5
{'LumA', 'LumB', 'Basal', 'Her2', 'Normal'}


## Now, sort the folder IDs into their subtypes

Notice that the order of folders_list is in the same order as match_subtypes

In [115]:
## uses index of subtype label to sort folder IDs into subtypes

# Luminal A samples
lumA_samples = list()
for i in range(0,len(match_subtypes)):
    if match_subtypes[i]=="LumA":
        lumA_samples.append(folders_list[i])

# Luminal B samples
lumB_samples = list()
for i in range(0,len(match_subtypes)):
    if match_subtypes[i]=="LumB":
        lumB_samples.append(folders_list[i])

# Basal samples
basal_samples = list()
for i in range(0,len(match_subtypes)):
    if match_subtypes[i]=="Basal":
        basal_samples.append(folders_list[i])

# Her2 samples
her2_samples = list()
for i in range(0,len(match_subtypes)):
    if match_subtypes[i]=="Her2":
        her2_samples.append(folders_list[i])

# Normal samples
norm_samples = list()
for i in range(0,len(match_subtypes)):
    if match_subtypes[i]=="Normal":
        norm_samples.append(folders_list[i])

In [116]:
print(len(lumB_samples))

126


In [117]:
# gets list of genes that are in the file

geneList = list() # list of all genes that are noted in the expression files

with open(dataNewFolder + '/' + indivList[0], "r") as file:
    for line in file.readlines():
        potentialGene = line.strip().split("\t")[0]
        
        if potentialGene[0:3] == "ENS":
            newGeneName = potentialGene.strip().split(".")[0]
            geneList.append(newGeneName)
print(len(geneList))

geneList_lumA = geneList
geneList_lumB = geneList
geneList_basal = geneList
geneList_her2 = geneList
geneList_norm = geneList

60483


In [118]:
## Reads through the unzipped expression files and creates the expression matrices depending on how the file
# name matches with the folder IDs of each subtype

lumA_exp = np.zeros((len(geneList), len(lumA_samples))) # exp matrix of LumA samples
lumB_exp = np.zeros((len(geneList), len(lumB_samples))) # exp matrix of LumB samples
basal_exp = np.zeros((len(geneList), len(basal_samples))) # exp matrix of basal samples
her2_exp = np.zeros((len(geneList), len(her2_samples))) # exp matrix of her2 samples
norm_exp = np.zeros((len(geneList), len(norm_samples))) # exp matrix of LumA samples

print(lumA_exp.shape) # just to make sure dimensions are correct
 
lumA_j = 0 # index of the column (sample)index of the column (sample)
lumB_j = 0
basal_j = 0
her2_j = 0
norm_j = 0
for fileName in os.listdir(dataNewFolder):
    if fileName == '.DS_Store' or fileName == 'MANIFEST.txt':
        continue
    
    if fileName in indivList:
        
        temp_folder = folderIDs[indivList.index(fileName)]
        readFile = dataNewFolder + '/' + fileName
            
        # adds to luminal A expression matrix
        if temp_folder in lumA_samples:
            
            lumA_i = 0
            with open(readFile, "r") as file:
                for line in file.readlines():
                    if("no_feature" in line):
                        break
                    lumA_exp[lumA_i, lumA_j] = float(line.strip().split("\t")[1])
                    lumA_i += 1
                    
                lumA_j += 1
                        
        # adds to luminal B expression matrix
        if temp_folder in lumB_samples:
            
            lumB_i = 0
            with open(readFile, "r") as file:
                for line in file.readlines():
                    if("no_feature" in line):
                        break
                    lumB_exp[lumB_i, lumB_j] = float(line.strip().split("\t")[1])
                    lumB_i += 1
                    
                lumB_j += 1
        
        # adds to basal expression matrix
        if temp_folder in basal_samples:
            
            basal_i = 0
            with open(readFile, "r") as file:
                for line in file.readlines():
                    if("no_feature" in line):
                        break
                    basal_exp[basal_i, basal_j] = float(line.strip().split("\t")[1])
                    basal_i += 1
                    
                basal_j += 1
        
        # adds to her2 expression matrix
        if temp_folder in her2_samples:
            
            her2_i = 0
            with open(readFile, "r") as file:
                for line in file.readlines():
                    if("no_feature" in line):
                        break
                    her2_exp[her2_i, her2_j] = float(line.strip().split("\t")[1])
                    her2_i += 1
                    
                her2_j += 1
        
        # adds to norm expression matrix
        if temp_folder in norm_samples:
            
            norm_i = 0
            with open(readFile, "r") as file:
                for line in file.readlines():
                    if("no_feature" in line):
                        break
                    norm_exp[norm_i, norm_j] = float(line.strip().split("\t")[1])
                    norm_i += 1
                    
                norm_j += 1
                    

(60483, 423)


In [119]:
## Makes a giant expression matrix for TMM norm in R
# makes a matrix where patients are columns and genes are rows

all_exp = np.zeros((len(geneList), len(indivList))) # matrix with all info from subtypes
j = 0 # index of the column (sample)
i = 0 # index of the row (gene)
print(all_exp.shape) # just to make sure dimensions are correct

for fileName in os.listdir(dataNewFolder):
    if fileName == '.DS_Store' or fileName == 'MANIFEST.txt':
        continue
        
    if fileName in indivList:
        temp_folder = folders_list[indivList.index(fileName)]
        readFile = dataNewFolder + '/' + fileName
    
        with open(readFile, "r") as file:
            for line in file.readlines():
                if("no_feature" in line):
                        break
                all_exp[i, j] = float(line.strip().split("\t")[1])
                i+=1
                
            i = 0
            if j != len(indivList):
                j+=1


(60483, 832)


In [120]:
print(all_exp[0:5,])

[[2.527e+03 1.397e+03 4.948e+03 ... 3.145e+03 6.713e+03 4.680e+02]
 [3.000e+00 2.000e+00 2.100e+01 ... 4.600e+01 5.690e+02 3.000e+00]
 [2.413e+03 2.021e+03 1.668e+03 ... 3.219e+03 2.248e+03 1.825e+03]
 [3.410e+03 2.140e+03 2.220e+03 ... 2.521e+03 2.452e+03 1.494e+03]
 [1.281e+03 7.480e+02 6.120e+02 ... 9.190e+02 5.160e+02 5.970e+02]]


In [124]:
## Makes the expression file

PANDAexpPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/all_exp_HTSeq.txt'
geneExpFile = open(PANDAexpPath,"w+")

large_string = match_barcodes[0]
for f in range(1, len(match_barcodes)):
    large_string = large_string + "\t" + match_barcodes[f]
    
newLine = large_string + "\n"
geneExpFile.write(newLine)

for i in range(0, len(geneList)):
    newLine = geneList[i]
    
    for j in range(0, len(file_id)):
        newLine = newLine + "\t" + str(all_exp[i,j])
        
    newLine = newLine + "\n"
    geneExpFile.write(newLine)
    
geneExpFile.close()

In [125]:
## Makes a reference file of file names and their subtypes

ref_col_path = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/all_labels.txt'
refFile = open(ref_col_path,"w+")

for i in range(0, len(match_barcodes)):
    ref_row = match_barcodes[i] + "\t" + match_subtypes[i] + "\n"
    refFile.write(ref_row)
    
refFile.close()



In [126]:
print(len(set(match_barcodes)))

714
